<h1> CS753 Assignment 4 Recommender Systems</h1>



<h4>1. Reading json files: https://www.geeksforgeeks.org/read-json-file-using-python/ </h4>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pandas.api.types import CategoricalDtype
from scipy import sparse
import random
from scipy.stats import norm

In [2]:
# load data using Python JSON module
train = pd.read_json (r'...train.json', lines=True)
train['type']='train'
val = pd.read_json (r'...val.json', lines=True)
val['type']='val'
test = pd.read_json (r'...test.json', lines=True)
test['type']='test'
print ("No. of train: ", len(train), "No. of validation: ", len(val), "No. of test: ", len(test))

# obtain unique values of business_id and user_id in all dataset
all = pd.concat([train,val,test])
all_user = all['user_id'].unique()
all_business = all['business_id'].unique()
shape =(len(all_user),len(all_business))
print("shape of sparseMatrix: ", shape, "; total unique user_id: ",len(all['user_id'].unique()), 
      "; total unique business_id: ", len(all['business_id'].unique()))
# Create indices for users and businesses in train dataset
user_cat = CategoricalDtype(categories=sorted(all_user), ordered=True)
business_cat = CategoricalDtype(categories=sorted(all_business), ordered=True)
user_index_train = train["user_id"].astype(user_cat).cat.codes
business_index_train = train["business_id"].astype(business_cat).cat.codes
user_index_val = val["user_id"].astype(user_cat).cat.codes
business_index_val = val["business_id"].astype(business_cat).cat.codes
user_index_test = test["user_id"].astype(user_cat).cat.codes
business_index_test = test["business_id"].astype(business_cat).cat.codes

# create sparse matrix for train, val and test
csc_train = sparse.csc_matrix((train["stars"], (user_index_train, business_index_train)), shape=shape)
csc_val = sparse.csc_matrix((val["stars"], (user_index_val, business_index_val)), shape=shape)
csc_test = sparse.csc_matrix((test["stars"], (user_index_test, business_index_test)), shape=shape)
print("shape of matrix train, val and test: " ,csc_train.shape,csc_test.shape,csc_val.shape)

No. of train:  841020 No. of validation:  8582 No. of test:  8582
shape of sparseMatrix:  (41701, 7965) ; total unique user_id:  41701 ; total unique business_id:  7965
shape of matrix train, val and test:  (41701, 7965) (41701, 7965) (41701, 7965)


<h4>1. Task 1: Estimate the global bias bg, user specific bias b(user)i and item specific bias b(item)j on the training data. Report the global bg, the user specific bias of the user with user id= “b4aIMeXOx4cn3bjtdIOo6Q” , item specific bias of the business with business id = “7VQYoXk3Tc8EZeKuXeixeg”. [10 pts] </h4>

In [3]:
# global bias bg,  user bias bi and item bias bj for training data

bg = csc_train.sum()/csc_train.count_nonzero()
bi = csc_train.sum(axis =1)/csc_train.astype(bool).sum(axis=1).sum()
bj = (csc_train.sum(axis =0)/csc_train.astype(bool).sum(axis=0).sum()).T

print("Global bias bg: ", round(bg,3))
#print(bi.shape, "user specific bias bi: ", bi)
#print(bj.shape, "item specific bias bj: ", bj)

# get index for specific user and business
bi_index = np.where(user_cat.categories == 'b4aIMeXOx4cn3bjtdIOo6Q')
bi_bias = np.squeeze(bi[[bi_index]]).item()
bj_index = np.where(business_cat.categories == '7VQYoXk3Tc8EZeKuXeixeg')
bj_bias = np.squeeze(bj[[bj_index]]).item()

#print(bi_index,bj_index)
print('The user specific bias of the user with user id= “b4aIMeXOx4cn3bjtdIOo6Q": ', round(bi_bias,3))
print('The item specific bias of the business with business id = “7VQYoXk3Tc8EZeKuXeixeg”: ', round(bj_bias,3))

Global bias bg:  3.829
The user specific bias of the user with user id= “b4aIMeXOx4cn3bjtdIOo6Q":  0.0
The item specific bias of the business with business id = “7VQYoXk3Tc8EZeKuXeixeg”:  0.0


C:\Users\ylive\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:193: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = N.ndarray.__getitem__(self, index)


<h4>2. Task 2: Implement the basic latent factor model without considering the bias: rij =qTi dot pj. Set the number of latent factors k = 8. Run Stochastic Gradient Descent (SGD) for 10 epoches with a fixed learning rate η = 0.01 and regularization hyperparameter λ1 = λ2 = 0.3. Report the RMSE on the training data for each epoch. [30 pts]</h4>

<h5> The result shows RMSE is reducing with the increasing of epoch number, it shows the error is approach local minima</h5>

In [4]:
## function for calculating RMSE
def GD_RMSE(RR,Q,P):
    rows,columns,values = sparse.find(RR)
    e = 0
    size = len(values)
    for i,j,v in zip(rows,columns,values):
        e= e + pow(v-np.dot(Q[i,:],P[j,:]),2)
    rmse_gd = np.sqrt(e/size)
    return(rmse_gd)

## function for stochastic gradient descent model
# R:csc sparse matrix; K:latent factor, lr: learning rate, lamda1 lamda2: hyperparameters
def SGD(R,K = 8, lr = 0.01, lamda1 =0.3, lamda2=0.3, epoch = 10 ): 
    rows,columns,values = sparse.find(R) 
    #  Latent Factor matrix shape
    M,N = R.shape
    # using random values of Q and P initialized with normal distribution at mean 0, SD 0.15 and 0.1 with values between [-1.1]
    Q=np.random.normal(0, 0.15, M*K).reshape((M,K)) # user factor matrix
    P=np.random.normal(0, 0.1, N*K).reshape((N,K)) # business factor matrix
    # calculate initial RMSE
    rmse1 = GD_RMSE(R,Q,P)
    print("Initial RMSE: ", round(rmse1,3))
    for e in range(0,epoch):
        for i,j,v in zip(rows, columns, values): # for each rating of user i and business j
            r_hat = np.dot(Q[i,:],P[j,:])           # get rating prediction
            for f in range(0,K):
                loss_q = -2*(v - r_hat)*P[j,f] + 2*lamda1 * Q[i,f]
                loss_p = -2*(v - r_hat)*Q[i,f] + 2*lamda2 * P[j,f]
                Q[i,f] -= lr * loss_q
                P[j,f] -= lr * loss_p
        rmse2 = GD_RMSE(R,Q,P)
        print("At Epoch ",e+1 ,", RMSE: ", round(rmse2,3))
    return(Q,P)
## Run stochastic gradient descent on training data for 10 epoches 
SGD_train = SGD(csc_train)

Initial RMSE:  4.044
At Epoch  1 , RMSE:  2.579
At Epoch  2 , RMSE:  1.324
At Epoch  3 , RMSE:  1.185
At Epoch  4 , RMSE:  1.156
At Epoch  5 , RMSE:  1.145
At Epoch  6 , RMSE:  1.14
At Epoch  7 , RMSE:  1.137
At Epoch  8 , RMSE:  1.135
At Epoch  9 , RMSE:  1.133
At Epoch  10 , RMSE:  1.132


<h4>3. Task 3: Use SGD to train the latent factor model with different values of k in {4, 8, 16}
and stop after 10 epoches. Report the RMSE for each value of k on the validation
data (“val.json”). Pick the model that results in the best RMSE on the validation
set and report its RMSE on the test data (“test.json”). [15 pts] </h4>

In [5]:
# R:csc sparse matrix; K:latent factor, lr: learning rate, lamda1 lamda2: hyperparameters
def SGD_train(R,K = 8, lr = 0.01, lamda1 =0.3, lamda2=0.3, epoch = 10 ): 
    rows,columns,values = sparse.find(R) 
    #  Latent Factor matrix shape
    M,N = R.shape
    # using random values of Q and P 
    Q=np.random.normal(0, 0.15, M*K).reshape((M,K)) # user factor matrix
    P=np.random.normal(0, 0.1, N*K).reshape((N,K)) # business factor matrix
    for e in range(0,epoch):
        for i,j,v in zip(rows, columns, values): # for each rating of user i and business j
            r_hat = np.dot(Q[i,:],P[j,:])           # get rating prediction
            for f in range(0,K):
                loss_q = -2*(v - r_hat)*P[j,f] + 2*lamda1 * Q[i,f]
                loss_p = -2*(v - r_hat)*Q[i,f] + 2*lamda2 * P[j,f]
                Q[i,f] -= lr * loss_q
                P[j,f] -= lr * loss_p
    return(Q,P)

## Using Q,P obtained from training data to calculate RMSE on validation data for k = [4,8,16]
## to choose the best k at minimum RMSE
result =[]
for i in [4,8,16]:
    result.append(i)
    sgdmatrix = SGD_train(csc_train, K = i)
    # get RMSE on validation data
    RMSE = GD_RMSE(csc_val,sgdmatrix[0],sgdmatrix[1])
    result.append(RMSE)
    print(" when k = ", i, " RMSE of validation dataset is :" , round(RMSE,3))


 when k =  4  RMSE is : 1.582
 when k =  8  RMSE is : 1.59
 when k =  16  RMSE is : 1.584


In [6]:
# keep result in 3X2 array
result_RMSE = np.asarray(result).reshape((3,2))
# get k of mininum RMSE
best_k = int(np.squeeze(result_RMSE[np.where(result_RMSE[:,1] ==result_RMSE[:,1].min(axis=0)),0]).item())
print(" The best RMSE is: ",round(result_RMSE[:,1].min(axis=0),3), " when k = ", best_k)

 The best RMSE is:  1.582  when k =  4


In [7]:
## Using best k for testing data (csc_test)
sgdmatrix_test =SGD_train(csc_train, K = best_k)
RMSE_test = GD_RMSE(csc_test,sgdmatrix_test[0],sgdmatrix_test[1])
print(" RMSE o test data is: ", round(RMSE_test,3))

 RMSE on test data is:  1.677


<h5> Result shows test RMSE is slight higher than the validation error and worse than the training data. This is expected. </h5>

<h4>4. Task 4: Incorporate the bias terms bg, b(user)i and b(item)jto the latent factor model:rij = bg + b(user)i + b(item)j + qTipj and learn the user bias and business bias from data.Initialize the user bias b(user)i and item bias terms b(item)j using the values computed in Task 1. Set the number of latent factors k = 8. Run SGD for 10 epoches with a fixed learning rate η = 0.01 and regularization hyperparameter λ1 = λ2 = λ3 = λ4 = 0.3.Report the RMSE on the training data for each epoch and the learned bias terms.[30 pts]</h4>

In [10]:
## function for calculating RMSE
def RMSE_bias(RR,Q,P,bg,bi,bj):
    rows,columns,values = sparse.find(RR)
    e = 0
    size = len(values)
    for i,j,v in zip(rows,columns,values):
        e= e + pow(v-np.dot(Q[i,:],P[j,:])-bg-bi[i,0]-bj[j,0],2)
    rmse = np.sqrt(e/size)
    return(rmse)
# SGD function with bias # R:csc sparse matrix; K:latent factor, lr: learning rate, lamda1,2,3,4: hyperparameters
def SGD_bias(R,K = 8, lr = 0.01, lamda1 =0.3, lamda2=0.3,lamda3 =0.3,lamda4=0.3, epoch = 10 ): 
    #  Latent Factor matrix shape
    M,N = R.shape
    # claculate bg, bi (item bias) and bj (business bias)
    bg = R.sum()/R.count_nonzero()
    bj_val = (R.sum(axis =0)/R.astype(bool).sum(axis=0).sum()).T
    bi_val = R.sum(axis =1)/R.astype(bool).sum(axis=1).sum()
    bi = np.asarray(bi_val).reshape((M,1))
    bj = np.asarray(bj_val).reshape((N,1))
    
    # get rows,columns,values of non-zero values
    rows,columns,values = sparse.find(R) 

    # using random values of Q and P 
    Q=np.random.normal(0, 0.15, M*K).reshape((M,K)) # user factor matrix
    P=np.random.normal(0, 0.1, N*K).reshape((N,K)) # business factor matrix
    # calculate initial RMSE
    rmse1 = RMSE_bias(R,Q,P,bg,bi,bj)
    print("Initial RMSE: ", round(rmse1,3))
    for e in range(0,epoch):
        for i,j,v in zip(rows, columns, values): # for each rating of user i and business j
            r_hat = np.dot(Q[i,:],P[j,:])           # get rating prediction
            ## calculate loss function for Q,P, bi and bj
            for f in range(0,K):
                loss_q = -2*(v - r_hat-bg-bi[i,0]-bj[j,0])*P[j,f] + 2*lamda1 * Q[i,f]
                loss_p = -2*(v - r_hat-bg-bi[i,0]-bj[j,0])*Q[i,f] + 2*lamda2 * P[j,f]
                loss_bi = -2*(v - r_hat-bg-bi[i,0]-bj[j,0]) + 2*lamda3 * bi[i,0]
                loss_bj = -2*(v - r_hat-bg-bi[i,0]-bj[j,0]) + 2*lamda4 * bj[j,0]
                Q[i,f] -= lr * loss_q
                P[j,f] -= lr * loss_p
                bi[i,0] -= lr* loss_bi
                bj[j,0] -= lr* loss_bj
        rmse2 = RMSE_bias(R,Q,P,bg,bi,bj)
        print("At Epoch ",e+1 ,", RMSE: ", round(rmse2,3))
    return(Q,P,bi,bj,bg)
SGDbias_train = SGD_bias(csc_train)
# get index for specific user and business
bg = SGDbias_train[4]
bi_index = np.where(user_cat.categories == 'b4aIMeXOx4cn3bjtdIOo6Q')
bi_bias_SGD = np.squeeze(SGDbias_train[2][bi_index,0]).item()
bj_index = np.where(business_cat.categories == '7VQYoXk3Tc8EZeKuXeixeg')
bj_bias_SGD = np.squeeze(SGDbias_train[3][bj_index,0]).item()
# claculate specific user bias by bi - bg, same for business bias by bj - bg
print('the user specific bias of the user with user id= “b4aIMeXOx4cn3bjtdIOo6Q": ', round(bi_bias_SGD - bg ,3))
print(' item specific bias of the business with business id = “7VQYoXk3Tc8EZeKuXeixeg”: ', round(bj_bias_SGD - bg ,3))

Initial RMSE:  1.3
At Epoch  1 , RMSE:  1.132
At Epoch  2 , RMSE:  1.128
At Epoch  3 , RMSE:  1.128
At Epoch  4 , RMSE:  1.128
At Epoch  5 , RMSE:  1.128
At Epoch  6 , RMSE:  1.128
At Epoch  7 , RMSE:  1.127
At Epoch  8 , RMSE:  1.127
At Epoch  9 , RMSE:  1.127
At Epoch  10 , RMSE:  1.127
the user specific bias of the user with user id= “b4aIMeXOx4cn3bjtdIOo6Q":  -3.957
 item specific bias of the business with business id = “7VQYoXk3Tc8EZeKuXeixeg”:  -3.626


<h4>5. Task 5: Similar to Task 3, find the best k on the validation set and apply the corresponding model to the test data. Compare the resulting test RMSE with Task3. [15 pts] </h4>
<h5> Result shows incooperating with rg,ri,rj would increase the prediction accuracy, comparing to GSD algorithm without bias loss correction. norm(ri) and norm(rj) work as a circle that regulate the loss function gradually reduce to within the circle thus to increase the prediction accuracy.

In [14]:
## modified SGD only print the final RMSE
def SGD_bias(R,K = 8, lr = 0.01, lamda1 =0.3, lamda2=0.3,lamda3 =0.3,lamda4=0.3, epoch = 10 ): # R:csc sparse matrix; K:latent factor, lr: learning rate, lamda1 lamda2: hyperparameters
    #  Latent Factor matrix shape
    M,N = R.shape
    bg = R.sum()/R.count_nonzero()
    bj_val = (R.sum(axis =0)/R.astype(bool).sum(axis=0).sum()).T
    bi_val = R.sum(axis =1)/R.astype(bool).sum(axis=1).sum()
    bi = np.asarray(bi_val).reshape((M,1))
    bj = np.asarray(bj_val).reshape((N,1))    
    # get rows,columns,values of non-zero values
    rows,columns,values = sparse.find(R) 
    # using random values of Q and P 
    Q=np.random.normal(0, 0.2, M*K).reshape((M,K)) # user factor matrix
    P=np.random.normal(0, 0.1, N*K).reshape((N,K)) # business factor matrix
    for e in range(0,epoch):
        for i,j,v in zip(rows, columns, values): # for each rating of user i and business j
            r_hat = np.dot(Q[i,:],P[j,:])           # get rating prediction
            ## calculate loss function for Q,P, bi and bj
            for f in range(0,K):
                loss_q = -2*(v - r_hat-bg-bi[i,0]-bj[j,0])*P[j,f] + 2*lamda1 * Q[i,f]
                loss_p = -2*(v - r_hat-bg-bi[i,0]-bj[j,0])*Q[i,f] + 2*lamda2 * P[j,f]
                loss_bi = -2*(v - r_hat-bg-bi[i,0]-bj[j,0]) + 2*lamda3 * bi[i,0]
                loss_bj = -2*(v - r_hat-bg-bi[i,0]-bj[j,0]) + 2*lamda4 * bj[j,0]
                Q[i,f] -= lr * loss_q
                P[j,f] -= lr * loss_p
                bi[i,0] -= lr* loss_bi
                bj[j,0] -= lr* loss_bj
    return(Q,P,bi,bj,bg)

## calculate RMSE on validation data for k = [4,8,16] to choose the best k at minimum RMSE
result =[]
for i in [4,8,16]:
    result.append(i)
    sgdmatrix = SGD_bias(csc_train, K = i)
    RMSE = RMSE_bias(csc_val,sgdmatrix[0],sgdmatrix[1],sgdmatrix[4],sgdmatrix[2],sgdmatrix[3])
    result.append(RMSE)
    print(" when k = ", i, " RMSE of validation dataset is :" , round(RMSE,3))
result_RMSE = np.asarray(result).reshape((3,2))
best_k = int(np.squeeze(result_RMSE[np.where(result_RMSE[:,1] ==result_RMSE[:,1].min(axis=0)),0]).item())
print(" The best RMSE is: ", round(result_RMSE[:,1].min(axis=0),3), " when K = ", best_k )

 when k =  4  RMSE of validation dataset is : 1.244
 when k =  8  RMSE of validation dataset is : 1.27
 when k =  16  RMSE of validation dataset is : 1.314
 The best RMSE is:  1.244  when K =  4


<h5> Final RMSE shows the training error after 10 epoches, RMSE shows the validation error at k = 4,8 and 16. Results show the error is minimum at k equals to 4. Hence we use K =4 for computing test errors.</h5>

In [15]:
## calculate RMSE on test data for k = best_k
sgdbias_test =SGD_bias(csc_train, K = best_k)
RMSE_test = RMSE_bias(csc_test,sgdbias_test[0],sgdbias_test[1],sgdbias_test[4],sgdbias_test[2],sgdbias_test[3])
print(" RMSE on test data is: ", round(RMSE_test,3))

 RMSE on test data is:  1.231
